In [2]:
import torch as th
from torch.optim import Adam

import gymnasium as gym

from gumbo.env.gym import TorchEnv
from gumbo.data.buffer import EpisodicBuffer
from gumbo.data.sampler import BatchSampler

from gumbo.modules.core import MLP
from gumbo.modules.encoder import IdentityEncoder, FlattenEncoder
from gumbo.modules.policy import DiagonalGaussianPolicy, CategoricalPolicy
from gumbo.modules.operator import ValueOperator

from gumbo.data.collector import Collector

from gumbo.optimizer import Optimizer

from gumbo.learn import PPO
from gumbo.learn import Trainer

In [3]:
env = gym.make("LunarLander-v2")
env = TorchEnv(env)

In [4]:
data = EpisodicBuffer.from_spec(env.env_spec, size=2048)

In [5]:
policy = CategoricalPolicy(
    encoder=FlattenEncoder(),
    body=MLP()
).build(env.obs_spec, env.act_spec)

In [6]:
collector = Collector(env, policy, data)

In [7]:
critic = ValueOperator(
    encoder=FlattenEncoder(),
    body=MLP()
).build(env.obs_spec)

In [8]:
sampler = BatchSampler(64, 10)

In [9]:
optimizer = Optimizer(Adam, lr=3e-4)

In [10]:
ppo = PPO(policy, critic, sampler, optimizer, ent_coef=0.01)

In [11]:
trainer = Trainer(collector, ppo)

In [ ]:
trainer.train(1000000)

In [18]:
data.episodes[0]._data

In [19]:
data

In [15]:
test = data.serve()

In [16]:
test.episodes[0]._data

In [17]:
test